# BOT PROJECT
#### NOTEBOOK AIM
This notebook initially acts as notebook / script 2 in the below BOT METHOD #1. See below for details.

### BOT METHOD #1
**Notebook / Script 1:**
1. Call in a websocket price stream from coinbase pro. 
2. Every new minute send close/open (seconds==0) price to a pickle file (dataframe or dictionary)
3. Set a high low price for the given minute and adjust it as the stream sets new records. The low / high value should be reset to the 1m candle open price each minute
4. Every time the high/low changes send the new price to a pickle file


**Notebook / Script 2:** 
1. Calls in the passed close/open price every minute to calculate and construct a supertrend dataframe 
2. So the above will be the close of the last candle and the open of the current candle
3. Also calls in the low high value passed by notebook 1. 
4. Intra-minute the low high will fluctuate as new records are set
5. Handles the connection to trade platform
6. Executes buy / sell orders on supertrend
7. Executes take profits / stops base on the latest high low values passes from the coinbase websocket

**Possible Problems**
1. The extra time required to pass new high lows from one script to the next could mean a loss of trade value
2. We don't know the problem of having two connections open at once - is this possible even from separate notebooks.
3. Need to solve how to update the supertrend with a dataframe updating every minute

## METHOD 2.1
IN the above method we read in a base 1m data frame from coinbase API and calculate a short history of supertrend indicator values. We then update the data set in real time via a websocket in another notebook. The problem here is that we haven't figured out how to stabilise this live update from the websocket. This results in small errors which affect the accuracy of price and supertrend data... this is not acceptable as we will end up with false trade signals. In addition once an error as occurred it has a knock on effect to all future data. Currently I can't see a way to manage this in an automated environment... or in reality, I think there may be an easier way to investigate before tackling that problem.

#### EASIER WAY?
What if we called the coinbase API every minute and calculated the supertrend from this accurate data. If a signal flashes we then monitor the data incoming from the websocket to check if targets / stops have been reached in real time. In addition we continue to call the API every minute to check the supertrend signal is still live or if it has been reversed!

#### PLAN
1. See how we can call the api at seconds ==0 every minute.
2. Calculate the supertrend and display the updated dataframe
3. Look for a supertrend signal

In [4]:
# Import libraries
import pandas as pd
import time
import matplotlib.pyplot as plt
import pandas_ta as ta
import talib
from datetime import datetime, timedelta, timezone
import requests
import numpy as np


from IPython.display import display, HTML

# Import Kucoin api libraries
from kucoin_futures.client import Market, Trade

# Import API key/secret
from api import *

# Set up the clien
client = Market(url='https://api-futures.kucoin.com')

#### BUILD THE API CALLER

In [5]:
def call_store():
    """
    In this function we call in the last hours worth of 1 minute data from the coinbase pro api.
    """
    
    api_url = 'https://api.pro.coinbase.com'
    symbol = 'BTC-USD'

    bar_size = "60"

    time_end = datetime.now()

    delta = timedelta(minutes=5)
    time_start = time_end - (60*delta)

#     print(time_end, time_start)

    # Put into isoformat for coinbase
    time_start = time_start.isoformat()
    time_end = time_end.isoformat()

    parameters = {
        "start":time_start,
        "end":time_end,
        "granularity": bar_size
    }

    btc_req = requests.get(f"{api_url}/products/{symbol}/candles", 
                         params=parameters, 
                         headers= {"content-type":"application/json"})

    btc = pd.DataFrame(btc_req.json(), 
                      columns=["time","low", "high","open","close","volume"]
                     )
    
    # re-order the features to standard OHLC
    btc = btc[["time","open", "high","low","close","volume"]]
    btc.time = pd.to_datetime(btc["time"], unit='s')

    # invert the called in data to carry out supertrend calcs
    btc = btc.loc[::-1]

    # Build the supertrend based on the trading view default parameters
    supertrendi = ta.supertrend(btc["high"],btc["low"], btc["close"], length=10, multiplier=3)

    # Rename the supertrend feature headers for ease of review
    supertrendi.rename(columns={'SUPERT_10_3.0': 'supertrend', 'SUPERTd_10_3.0': 'direction', 
                                'SUPERTl_10_3.0': 'st_long', 'SUPERTs_10_3.0': 'st_short'
                               }, inplace=True)

    btc = btc.join(supertrendi)
    btc.reset_index(inplace=True, drop=True)
    
    
    return btc

In [6]:
btc=call_store()
btc

,time,open,high,low,close,volume,supertrend,direction,st_long,st_short
0,2022-05-26 06:29:00,29602.91,29602.91,29546.73,29561.23,11.727724,0.000000,1,NaN,NaN
1,2022-05-26 06:30:00,29561.20,29563.22,29543.43,29553.13,5.238012,29442.242895,1,29442.242895,NaN
2,2022-05-26 06:31:00,29553.13,29553.13,29524.82,29524.82,12.403934,29442.242895,1,29442.242895,NaN
3,2022-05-26 06:32:00,29524.81,29550.33,29503.60,29550.33,13.989045,29442.242895,1,29442.242895,NaN
4,2022-05-26 06:33:00,29548.63,29568.65,29544.22,29567.92,8.140042,29453.357145,1,29453.357145,NaN
...,...,...,...,...,...,...,...,...,...,...
235,2022-05-26 10:24:00,29185.23,29185.76,29174.42,29181.68,2.988240,29127.818480,1,29127.818480,NaN
236,2022-05-26 10:25:00,29181.63,29183.23,29171.51,29171.51,2.322045,29127.818480,1,29127.818480,NaN
237,2022-05-26 10:26:00,29171.58,29202.99,29170.16,29196.39,5.838531,29127.818480,1,29127.818480,NaN
238,2022-05-26 10:27:00,29196.39,29196.39,29170.82,29172.85,4.645844,29127.818480,1,29127.818480,NaN


#### Set Signal and Trigger Functions 
These will allow us to identify supertrend signal flashes!

In [7]:
# Buy / Sell signal feature 
# If uptrend>0 then buy, elif downtrend>0 then sell, else: NAN

def signal(row):
    """
    In this function we'll build the feature that states whether we're in a buy or sell signal
    """
    
    uptrend = row[8]
    downtrend = row[9]


    if uptrend>0:
        return "buy"
    elif downtrend>0:
        return "sell"
    else:
        return "no_signal"
    


def trigger(row):
    global idx
    
    signal = row[10]
    
    if idx == 0:
        idx+=1
        return np.nan
    
    elif signal=="sell" and df.loc[idx-1].signal!="sell":
        idx+=1
        return "sellsig"

    elif signal=="buy" and df.loc[idx-1].signal!="buy":
        idx+=1
        return "buysig"
    
    else:
        idx+=1
        return "nosig"

#### SET AN  ORDER FUNCTION

In [8]:
order_record = pd.DataFrame(columns=["time_open", "rec_open", "direction", "entry_price", "bid",
                                     "ask", "tp", "rec_close", "act_close", "net_return", "time_closed"
                                    ])

def order_handle(trade, rec_open, order_state, time, tp):
    
    """
    In this function we execute and close orders
    """
    global order_record
    
    btcusd = client.get_ticker('XBTUSDM')
    live_price = float(btcusd["price"])
    bid = float(btcusd["bestBidPrice"])
    ask = float(btcusd["bestAskPrice"])
    tp = tp

    # If we have a new sell signal and ex trade still open - record details, close old buy & open new sell trade
    if order_state=="sell" and trade=="buy":
        tp = rec_open*1.0058
        # Calculate closed sell trade returns
        order_record.iat[-1,7]=rec_open
        order_record.iat[-1,8]=live_price
        order_record.iat[-1,9]=order_record.iloc[-1].entry_price-order_record.iloc[-1].act_close
        order_record.iat[-1,10]=time
        
        order_record = order_record.append({"time_open":time, "rec_open":rec_open, "direction":trade, 
                                            "entry_price":live_price, "bid":bid, "ask":ask, "tp":tp}, ignore_index=True)
        print(f"Reversal {trade} order sent")
    
    # If we have a new buy signal and ex trade still open - record details, close old sell & open new buy trade  
    elif order_state=="buy" and trade=="sell":
        tp = rec_open*0.9942
        # Calculate closed buy trade returns
        order_record.iat[-1,7]=rec_open
        order_record.iat[-1,8]=live_price
        order_record.iat[-1,9]=order_record.iloc[-1].act_close-order_record.iloc[-1].entry_price
        order_record.iat[-1,10]=time
        
        order_record = order_record.append({"time_open":time, "rec_open":rec_open, "direction":trade, 
                                            "entry_price":live_price, "bid":bid, "ask":ask, "tp":tp}, ignore_index=True)
        print(f"Reversal {trade} order sent")
    
    # If we have no open orders and recieve a new trade signal - open a new trade
    elif order_state=="closed" and (trade=="buy" or trade=="sell"):
        # Create a new order row because our last trade has been closed prior
        order_record = order_record.append({"time_open":time, "rec_open":rec_open, "direction":trade, 
                                            "entry_price":live_price, "bid":bid, "ask":ask, "tp":tp, "rec_close":0, 
                                            "act_close":0, "net_return":0, "time_closed":0}, ignore_index=True)
        print(f"new {trade} order_sent")
    
    # If we have recieved the first trade signal in the session - open a new trade    
    elif order_state=="dormant": ###  dormant
        order_record = order_record.append({"time_open":time, "rec_open":rec_open, "direction":trade, 
                                            "entry_price":live_price, "bid":bid, "ask":ask, "tp":tp, "rec_close":0, 
                                            "act_close":0, "net_return":0, "time_closed":0}, ignore_index=True)
                                            
    
    # If an open buy trade has recorded a tp hit, record the trade details
    elif order_state=="closed" and trade=="buy_closed":
        print("trade record updated")
        order_record.iat[-1,7]=rec_open # close / tp price theoretical
        order_record.iat[-1,8]=live_price # close price achieved with exchange
        order_record.iat[-1,9]=order_record.iloc[-1].act_close-order_record.iloc[-1].entry_price # p/l
        order_record.iat[-1,10]=time # trade close time
        
    # If an open sell trade has recorded a tp hit, record the trade details
    elif order_state=="closed" and trade=="sell_closed":
        print("trade record updated")
        order_record.iat[-1,7]=rec_open # close / tp price theoretical
        order_record.iat[-1,8]=live_price # close price achieved with exchange
        order_record.iat[-1,9]=order_record.iloc[-1].entry_price-order_record.iloc[-1].act_close # p/l
        order_record.iat[-1,10]=time # trade close time
        
        
        
    """
    To simplify coudn't we just merge the closed and dormant order_states? - we'd need to add the trade status
    to the dormant / closed consitional. TO BE RESEARCHED!!
    """
    
        
    return None
      

#### BUILD THE RECURRING MINUTELY DF BUILDER

In [9]:
called = False
idx=0
order_sent=False
order_state="dormant"
tp = 0
live_check=False
while True:
    call_time = datetime.now().second
#     if call_time<10:
#         print(call_time)
#     elif call_time>55:
#         print("Minute almost over")

    # If a new minute just started - get the data and look for a signal - give the api 1 second to resolve the 
    # last minute then call
    if call_time==1 and called==False:
        df = call_store()
        
        
        # Call the signal function and join to the base data
        signal_ = df.apply(signal, axis='columns').to_frame()
        df = df.join(signal_)
        df.rename(columns={0: 'signal'}, inplace=True)

        # Call the trigger function and join to the master data
        idx=0 # We have to reset our idx counter before running this apply function
        trigger_ = df.apply(trigger, axis="columns").to_frame()
        df = df.join(trigger_)
        df.rename(columns={0: 'trigger'}, inplace=True)
        # Get the last two rows to display
        dft = df.tail(2)
        
        # display the last few rows of the updated dataframe
        display(HTML(dft.to_html()))
        called=True
        
        order_sent=False
        
        if df.iloc[-1].trigger=="buysig":
            tp=df.iloc[-1].close*1.0058
            order_handle("buy", df.iloc[-1].close, order_state=order_state, time=df.iloc[-1].time, tp=tp)
            print("buy order sent!")
            order_state="buy"
            display(HTML(order_record.to_html()))
            
            
        elif df.iloc[-1].trigger=="sellsig":
            tp=df.iloc[-1].close*0.9942
            order_handle("sell", df.iloc[-1].close, order_state=order_state, time=df.iloc[-1].time, tp=tp)
            print("sell order sent!")
            order_state="sell"
            display(HTML(order_record.to_html()))
            
            
        
    if call_time>45 and called==True:
        called=False
      
    # Now we want to look at the live price - every 2 seconds - and see if our tp has been hit!
    if order_state == "buy" or order_state == "sell":
        if call_time%2==0 and live_check==False:
            ## Look at the live price
            live_check=True
            try:
                live_data = pd.read_csv("prices_test.csv")
            except:
                continue
            # live_data.time = pd.to_datetime(live_data["time"])
            if order_state=="buy" and live_data.iloc[-1].high>=tp:
                order_state="closed"
                order_handle(trade="buy_closed", rec_open=tp, order_state=order_state, time=datetime.utcnow(), tp=tp)
                print("attempting buy order closure...")
                display(HTML(order_record.to_html()))
                
            elif order_state=="sell" and live_data.iloc[-1].low<=tp:
                order_state="closed"
                order_handle(trade="sell_closed", rec_open=tp, order_state=order_state, time=datetime.utcnow(), tp=tp)
                print("attempting sell order closure")
                display(HTML(order_record.to_html()))
                
                
                
    if call_time%2==1 and live_check==True:
        live_check=False
                

        

,time,open,high,low,close,volume,supertrend,direction,st_long,st_short,signal,trigger
237,2022-05-26 10:27:00,29196.39,29196.39,29170.82,29172.85,4.645844,29127.81848,1,29127.81848,NaN,buy,nosig
238,2022-05-26 10:28:00,29172.30,29173.12,29157.88,29160.76,10.081030,29127.81848,1,29127.81848,NaN,buy,nosig


,time,open,high,low,close,volume,supertrend,direction,st_long,st_short,signal,trigger
237,2022-05-26 10:28:00,29172.30,29173.12,29157.88,29160.76,10.081030,29127.81848,1,29127.81848,NaN,buy,nosig
238,2022-05-26 10:29:00,29160.57,29169.65,29156.20,29164.99,5.707511,29127.81848,1,29127.81848,NaN,buy,nosig


,time,open,high,low,close,volume,supertrend,direction,st_long,st_short,signal,trigger
237,2022-05-26 10:29:00,29160.57,29169.65,29156.20,29164.99,5.707511,29127.81848,1,29127.81848,NaN,buy,nosig
238,2022-05-26 10:30:00,29164.99,29203.06,29161.27,29200.75,5.905291,29127.81848,1,29127.81848,NaN,buy,nosig


,time,open,high,low,close,volume,supertrend,direction,st_long,st_short,signal,trigger
237,2022-05-26 10:30:00,29164.99,29203.06,29161.27,29200.75,5.905291,29127.81848,1,29127.81848,NaN,buy,nosig
238,2022-05-26 10:31:00,29200.69,29203.68,29188.08,29190.67,4.160443,29127.81848,1,29127.81848,NaN,buy,nosig


,time,open,high,low,close,volume,supertrend,direction,st_long,st_short,signal,trigger
237,2022-05-26 10:31:00,29200.69,29203.68,29188.08,29190.67,4.160443,29127.818480,1,29127.818480,NaN,buy,nosig
238,2022-05-26 10:32:00,29190.67,29214.23,29190.66,29190.66,4.078344,29133.867168,1,29133.867168,NaN,buy,nosig


...

,time,open,high,low,close,volume,supertrend,direction,st_long,st_short,signal,trigger
237,2022-05-26 17:40:00,29561.34,29566.0,29555.5,29565.15,12.492476,29469.468672,1,29469.468672,NaN,buy,nosig
238,2022-05-26 17:41:00,29564.65,29566.0,29549.7,29562.39,18.141045,29470.806805,1,29470.806805,NaN,buy,nosig


,time,open,high,low,close,volume,supertrend,direction,st_long,st_short,signal,trigger
237,2022-05-26 17:41:00,29564.65,29566.00,29549.70,29562.39,18.141045,29470.806805,1,29470.806805,NaN,buy,nosig
238,2022-05-26 17:42:00,29562.25,29574.83,29548.68,29559.51,21.411086,29475.571124,1,29475.571124,NaN,buy,nosig


,time,open,high,low,close,volume,supertrend,direction,st_long,st_short,signal,trigger
237,2022-05-26 17:42:00,29562.25,29574.83,29548.68,29559.51,21.411086,29475.571124,1,29475.571124,NaN,buy,nosig
238,2022-05-26 17:43:00,29559.51,29578.77,29553.98,29578.76,13.870803,29481.372512,1,29481.372512,NaN,buy,nosig


,time,open,high,low,close,volume,supertrend,direction,st_long,st_short,signal,trigger
237,2022-05-26 17:43:00,29559.51,29578.77,29553.98,29578.76,13.870803,29481.372512,1,29481.372512,NaN,buy,nosig
238,2022-05-26 17:44:00,29581.65,29628.72,29572.83,29607.16,30.968896,29507.505761,1,29507.505761,NaN,buy,nosig


KeyboardInterrupt: 

In [ ]:
30358-30222

In [3]:
live_data = pd.read_csv("prices_test.csv")
live_data.iloc[-1].high, live_data.iloc[-1].low
live_data

,time,open,high,low,close
0,2022-05-25 17:03:00.720267+00:00,29542.19,29545.23,29534.90,29538.82
1,2022-05-25 17:04:00.258194+00:00,29539.55,29546.58,29524.22,29537.12
2,2022-05-25 17:05:00.033476+00:00,29537.12,29550.00,29520.12,29534.93
3,2022-05-25 17:07:00.123677+00:00,29534.93,29555.60,29534.04,29564.39
4,2022-05-25 17:08:00.323900+00:00,29565.00,29574.99,29546.26,29565.00


## DEBUG NOTES
1. On signal flash we need to send the tp to the order handler trade dataframe ** DONE **
2. Change the feature order to OHLCV ** DONE **
3. Even with a 2 second delay we are occassionaly getting incorrect close price records. This resulted in a sell trade at 11:09 where the recorded priced created a trigger. This was resolved with a true sell sig at 11:10 but we need to understand how best to avoid this... WE NOTE THIS HAPPENS MORE OFTEN THAN THOUGHT. 
    - extend the delay to 3 seconds, 
    - make a retrospective check 1m later to redress any signal errors?
    - what happens if we just leave the current code to its own devices?
4. The code does miss / slip some tp hits eg at 13:25 our price hit the tp but did not register in the live environment so the trade did not close. Profit is missed here. ** FIX MADE - MONITOR **
5. The live code doesn't display the order_record dataset when a trade is closed - so we cant't see results in real time. Alter the code so we can see the trade has closed!! ** DONE - MONITOR **
6. Maybe add a nice running total p&l total print out for interest!
7. The order_record is recording the wrong close price... it needs to record the tp as the rec_close and the live price as the actual close. At the minute it is recording the 1m close as the rec_close price - this is wrong!??

In [55]:
strtm = f"{datetime.utcnow():%Y-%m-%d %H:%M:%S}"
pd.to_datetime(strtm)

Timestamp('2022-05-27 09:44:05')

## SANDPIT

In [ ]:
order_record

In [ ]:
tim = datetime.now()

In [21]:
printed=False
x=12
while True:

    if datetime.now().second%5==0 and printed==False:
        print(x)
        x=x+2
        printed=True
    elif datetime.now().second%5!=0 and printed==True:
        printed=False

12
14


KeyboardInterrupt: 

In [ ]:
order_record.append({"time_open":tim, "rec_open":9, "direction":"buy", "price":9, 
                                            "bid":9, "ask":12, "tp":7, "rec_close":0, "act_close":0, 
                                            "net_return":0, "time_closed":0}, ignore_index=True)

In [ ]:
btcusd = client.get_ticker('XBTUSDM')
float(btcusd["price"])

In [ ]:
order_record